In [ ]:
import PySimpleGUI as sg
import vlc
from time import sleep
import os
from sys import platform

#----------- TEST MEDIA -----------#
#  next: playlist
import test_data
song1 = test_data.song1 # test a local path
song2 = test_data.song2 # test a remote path 

#----------- GUI -----------#
def layout(number):
    # can't use the same layout each time, but if the function takes an unused parameter, it works
    return [
        [sg.Graph(canvas_size=(1280,720), graph_bottom_left=(0,0), graph_top_right=(0,0), key='-G-')],
        [sg.Button('Exit', size=(10,1), pad=(0,0))]]

window = []

for i in range(8):
    window[i] = sg.Window('window'+str(i), layout(i), finalize=True, \
                    location=(0,0))


# window1 = sg.Window('111', layout(1), finalize=True, \
#                     location=(0,0))
# window2 = sg.Window('222', layout(2), finalize=True,
#                     location=(window1.current_location()[0]+1280,window1.current_location()[1]))
# window3 = sg.Window('333', layout(3), finalize=True,
#                     location=(window1.current_location()[0]+0,window1.current_location()[1]=720))


#----------- PLAYER SETUP -----------#
# shorten this code will need as many as 8 players
instance_list = []
for i in range(8):
    instance_list.append(vlc.Instance())
player_list = []
for i in range(8):
    player_list.append(instance_list[i].media_player_new())
media_list = []
for i in range(8):
    media_list.append(instance_list[i].media_new(song1))
for i in range(8):
    player_list[i].set_hwnd(window[i]['-G-'].Widget.winfo_id())
for i in range(8):
    player_list[i].set_media(media_list[i])



# instance1 = vlc.Instance()
# instance2 = vlc.Instance()
# player1 = instance1.media_player_new()
# player2 = instance2.media_player_new()
# media1 = instance1.media_new(song1)
# media2 = instance2.media_new(song2)
# player1.set_hwnd(window1['-G-'].Widget.winfo_id())
# player2.set_hwnd(window2['-G-'].Widget.winfo_id())
# player1.set_media(media1)
# player2.set_media(media2)


#----------- MAIN LOOP -----------#
# next: make sure media is on loop

i, paused = 0, [False, False]

while True:
    window, event, values = sg.read_all_windows(timeout=1000)
    for i in range(8):
        player_list[i].play()
        if (window[i] == sg.WIN_CLOSED) or (event == 'Exit'):
            break
    for i in range(8):
        window[i].close()

    # player1.play()
    # player2.play()
    # if (window1 == sg.WIN_CLOSED or window2 == sg.WIN_CLOSED) \
    #     or (event == 'Exit'):
    #     break
# window1.close()
# window2.close()

#----------- CLEANUP -----------#
# player1.stop()
# player2.stop()
# player1.release()
# player2.release()
# instance1.release()
# instance2.release()

#----------- OLD CODE -----------#
# # No.  Im not making this THAT robust, i need it finished
# def get_screen_size():
#     if platform == 'win32':
#         import win32api
#         return win32api.GetSystemMetrics(0), win32api.GetSystemMetrics(1)
#     else:
#         return 1920, 1080
# print(get_screen_size(), 'screen size')
# size1 = window1.size
# size2 = window2.size
# loc1 = window1.current_location()
# loc2 = window2.current_location()

